In [23]:
#------------------------------- PROBLEM 1B ----------------------------------#
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import time

# Check if GPU is available and set the device to GPU 1
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # Use the second GPU
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU.")

# Data loading
data_path = '../data-unversioned/p1ch7/'
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
])

cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=transform)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)

# Modified CNN Neural Network using nn.Sequential
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, kernel_size=3, padding=1),  # Additional Convolution Layer
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(64 * 8 * 8, 512),
    nn.ReLU(),
    nn.Linear(512, 10)
).to(device)  # Move the model to the specified device

# Training setup
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 300

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

overall_start_time = time.time()

for epoch in range(n_epochs):
    model.train()  # Set model to training mode
    total_train_loss = 0
    total_train_correct = 0
    total_train_samples = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)  # Move the data to the device

        outputs = model(imgs)
        loss = loss_fn(outputs, labels)
        total_train_loss += loss.item()

        _, predicted = torch.max(outputs, dim=1)
        total_train_correct += (predicted == labels).sum().item()
        total_train_samples += labels.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_train_loss / len(train_loader)
    train_accuracy = total_train_correct / total_train_samples

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    total_val_correct = 0
    total_val_samples = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)  # Move the data to the device

            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            total_val_loss += loss.item()

            _, predicted = torch.max(outputs, dim=1)
            total_val_correct += (predicted == labels).sum().item()
            total_val_samples += labels.size(0)

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = total_val_correct / total_val_samples

    print(f"Epoch: {epoch+1}, Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

overall_end_time = time.time()
total_training_time = overall_end_time - overall_start_time

print(f"Total training time: {total_training_time:.2f} seconds")




GPU not available, using CPU.


Files already downloaded and verified
Files already downloaded and verified
Epoch: 1, Training Loss: 1.8510, Training Accuracy: 0.3439, Validation Loss: 1.6492, Validation Accuracy: 0.4001
Epoch: 2, Training Loss: 1.4671, Training Accuracy: 0.4758, Validation Loss: 1.3888, Validation Accuracy: 0.4857
Epoch: 3, Training Loss: 1.3090, Training Accuracy: 0.5351, Validation Loss: 1.4386, Validation Accuracy: 0.5102
Epoch: 4, Training Loss: 1.2019, Training Accuracy: 0.5743, Validation Loss: 1.2063, Validation Accuracy: 0.5706
Epoch: 5, Training Loss: 1.1185, Training Accuracy: 0.6043, Validation Loss: 1.2173, Validation Accuracy: 0.5501
Epoch: 6, Training Loss: 1.0430, Training Accuracy: 0.6356, Validation Loss: 1.1634, Validation Accuracy: 0.5841
Epoch: 7, Training Loss: 0.9758, Training Accuracy: 0.6582, Validation Loss: 1.0214, Validation Accuracy: 0.6366
Epoch: 8, Training Loss: 0.9140, Training Accuracy: 0.6811, Validation Loss: 1.0721, Validation Accuracy: 0.6200
Epoch: 9, Training 